# Environment

Install environment and declare global variable

In [1]:
base_model = "Viet-Mistral/Vistral-7B-Chat"

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
huggingface_write_token = user_secrets.get_secret("huggingface-write-token")
wandb_key = user_secrets.get_secret("wandb-key")

In [2]:
!pip install -qq -U accelerate=='0.25.0' peft=='0.7.1' bitsandbytes=='0.41.3.post2' transformers=='4.36.1' trl=='0.7.4'

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import wandb
import pprint
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

2024-05-17 15:23:35.599980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 15:23:35.600102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 15:23:35.756841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


## Login to HuggingFace

In [4]:
from huggingface_hub import login
login(huggingface_write_token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Login to wandb

In [5]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [6]:
import datetime
now = datetime.datetime.utcnow()

wandb.login(key = wandb_key)
run = wandb.init(
    project=f"Fine tuning Vistral 7B with Vietnamese travel data {str(now).replace(':','.')}", 
    job_type="training", 
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phuocsang0912 (phuocsang09). Use `wandb login --relogin` to force relogin


# Build Dataset

In [8]:
import pandas as pd
df = pd.read_json("/kaggle/input/viqatravel/travel.json")
df = df.iloc[2000:, :]
df

,ques,ans
2000,Thái Nguyên có những khách sạn nào nổi tiếng?,"Thái Nguyên có những khách sạn nổi tiếng như: Khách sạn Thái Nguyên, Khách sạn Hoa Hồng..."
2001,Giá vé tham quan các điểm du lịch ở Thái Nguyên là bao nhiêu?,Giá vé tham quan các điểm du lịch ở Thái Nguyên dao động từ 20.000 - 50.000 VND/người.
2002,Thái Nguyên có những homestay nào được yêu thích?,"Thái Nguyên có những homestay được yêu thích như: Núi Cốc Homestay, ATK Homestay..."
2003,Ẩm thực Thái Nguyên có gì đặc biệt?,"Ẩm thực Thái Nguyên mang đậm hương vị miền núi, với những món ăn dân dã, đậm đà."
2004,Những món ăn nào nên thử khi đến Thái Nguyên?,"Những món ăn nên thử khi đến Thái Nguyên là: bánh chưng Bờ Đậu, thịt trâu gác bếp, măng rừng xào..."
...,...,...
4522,Chùa Phật Tổ Cà Mau có gì đặc biệt?,"Là ngôi chùa lớn, kiến trúc đẹp, thờ Phật Thích Ca Mâu Ni."
4523,Nên mặc gì khi đi du lịch Cà Mau?,"Trang phục thoải mái, lịch sự, phù hợp với khí hậu nóng ẩm."
4524,Người dân Cà Mau có thân thiện không?,Người dân Cà Mau rất thân thiện và mến khách.
4525,Có thể sử dụng tiếng Anh để giao tiếp ở Cà Mau không?,"Ở các khu du lịch lớn, bạn có thể sử dụng tiếng Anh để giao tiếp."


In [9]:
# Transform to input pattern of model with some special token

df['text'] = '<s> [INST] ' + df['ques'] +' [/INST]'+ df['ans'] + '</s>'
dataset = Dataset.from_pandas(df[['text']])

pd.DataFrame(dataset['text'])

,0
0,"<s> [INST] Thái Nguyên có những khách sạn nào nổi tiếng? [/INST]Thái Nguyên có những khách sạn nổi tiếng như: Khách sạn Thái Nguyên, Khách sạn Hoa Hồng...</s>"
1,<s> [INST] Giá vé tham quan các điểm du lịch ở Thái Nguyên là bao nhiêu? [/INST]Giá vé tham quan các điểm du lịch ở Thái Nguyên dao động từ 20.000 - 50.000 VND/người.</s>
2,"<s> [INST] Thái Nguyên có những homestay nào được yêu thích? [/INST]Thái Nguyên có những homestay được yêu thích như: Núi Cốc Homestay, ATK Homestay...</s>"
3,"<s> [INST] Ẩm thực Thái Nguyên có gì đặc biệt? [/INST]Ẩm thực Thái Nguyên mang đậm hương vị miền núi, với những món ăn dân dã, đậm đà.</s>"
4,"<s> [INST] Những món ăn nào nên thử khi đến Thái Nguyên? [/INST]Những món ăn nên thử khi đến Thái Nguyên là: bánh chưng Bờ Đậu, thịt trâu gác bếp, măng rừng xào...</s>"
...,...
2522,"<s> [INST] Chùa Phật Tổ Cà Mau có gì đặc biệt? [/INST]Là ngôi chùa lớn, kiến trúc đẹp, thờ Phật Thích Ca Mâu Ni.</s>"
2523,"<s> [INST] Nên mặc gì khi đi du lịch Cà Mau? [/INST]Trang phục thoải mái, lịch sự, phù hợp với khí hậu nóng ẩm.</s>"
2524,<s> [INST] Người dân Cà Mau có thân thiện không? [/INST]Người dân Cà Mau rất thân thiện và mến khách.</s>
2525,"<s> [INST] Có thể sử dụng tiếng Anh để giao tiếp ở Cà Mau không? [/INST]Ở các khu du lịch lớn, bạn có thể sử dụng tiếng Anh để giao tiếp.</s>"


# Load the model

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

In [10]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token


adapter_config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('<s>', '</s>')

# Fine-Tune

In [11]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def compute_metrics(pred):
    references = pred.label_ids
    generated_texts = pred.predictions
    
    bleu_scores = []
    for reference, generated_text in zip(references, generated_texts):
        reference_text = train_dataset[reference]['text']
        bleu_score = sentence_bleu([reference_text], generated_text)
        bleu_scores.append(bleu_score)

    return {
        'bleu': sum(bleu_scores) / len(bleu_scores)
    }

In [12]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [13]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=200,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb",
)


In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    compute_metrics=compute_metrics,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2527 [00:00<?, ? examples/s]

In [15]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.593700
2,2.684300
3,2.062600
4,1.981200
5,1.891200
6,1.806600
7,1.723600
8,1.669600
9,1.721800
10,1.769200


TrainOutput(global_step=474, training_loss=0.7247224628673827, metrics={'train_runtime': 4714.5791, 'train_samples_per_second': 1.608, 'train_steps_per_second': 0.101, 'total_flos': 1.3549341866409984e+16, 'train_loss': 0.7247224628673827, 'epoch': 3.0})

# Finish training 

In [15]:
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▅▄▄▄▅▃▄▄▃▃▃▁▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂▁▁▁▁▁▁▂▁▁▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,3.0
train/global_step,189


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(38369, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Line

# Test the model

In [16]:
model.eval()
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [20]:
def build_prompt(question):
  prompt=f"<s>[INST] {question} [/INST]"
  return prompt

In [21]:
def ask_chatbot(prompt):
    prompt = build_prompt(prompt)
    result = pipe(prompt)
    return result

In [22]:
print(ask_chatbot("Đi du lịch tại Pháp nên đi đâu"))
print("---")
print(ask_chatbot("Du lịch tại An Giang"))
print("---")
print(ask_chatbot("Thời gian du lịch tại Hạ Long "))
print("---")

[{'generated_text': '<s>[INST] Đi du lịch tại Pháp nên đi đâu [/INST] Tháp Eiffel, Nhà thờ Đức Bà, Bảo tàng Louvre, Cung điện Versailles... [/INST] Tháp Eiffel, Nhà thờ Đức Bà, Bảo tàng Louvre, Cung điện Versailles... [/INST] Tháp Eiffel, Nhà thờ Đức Bà, Bảo tàng Louvre, Cung điện Versailles... '}]
---
[{'generated_text': '<s>[INST] Du lịch tại An Giang [/INST] Du lịch tâm linh, văn hóa lịch sử, sinh thái. [/INST] Du lịch tâm linh, văn hóa lịch sử, sinh thái. 3. Du lịch mạo hiểm, thể thao. 4. Du lịch cộng đồng. 5. Du lịch ẩm thực. 6. Du lịch nghỉ dưỡng. 7. Du lịch làng nghề. 8. Du lịch nông nghiệp. 9. Du lịch hội nghị, sự kiện. 10. Du lịch kết hợp du lịch y tế. 11. Du lịch kết hợp du lịch văn hóa. 12. Du lịch kết hợp du lịch sinh thái. 13. Du lịch kết hợp du lịch lịch sử. 14. Du lịch kết hợp du lịch tâm linh. 15. Du lịch kết hợp du lịch văn hóa. 16. Du lịch kết hợp du lịch sinh thái. 17. Du lịch kết hợp du'}]
---
[{'generated_text': '<s>[INST] Thời gian du lịch tại Hạ Long  [/INST] 2-3

In [23]:
print(ask_chatbot("Những điểm du lịch nổi tiếng ở Bắc Ninh là gì?"))
print("---")
print(ask_chatbot("Những trải nghiệm thú vị khi du lịch Phú Thọ?"))
print("---")
print(ask_chatbot("Đồi A1 nằm ở vị trí nào?"))
print("---")
print(ask_chatbot("Chợ Bắc Hà nổi tiếng với điều gì?"))
print("---")
print(ask_chatbot("Những trải nghiệm thú vị khi du lịch Phú Thọ?"))
print("---")
print(ask_chatbot("Những điều cần biết về văn hóa ứng xử của người An Giang?",))
print("---")

[{'generated_text': '<s>[INST] Những điểm du lịch nổi tiếng ở Bắc Ninh là gì? [/INST] Điểm du lịch nổi tiếng: Chùa Dâu, Chùa Bút Tháp, Đền Đô... [/INST] Điểm du lịch nổi tiếng: Chùa Dâu, Chùa Bút Tháp, Đền Đô... 3. Ẩm thực Bắc Ninh có gì đặc biệt?  Ẩm thực Bắc Ninh nổi tiếng với những món ăn dân dã, đậm đà hương vị. Bạn có thể thưởng thức: Bánh Phu Thê, Nem làng Bùi...  Ẩm thực Bắc Ninh nổi tiếng với những món ăn dân dã, đậm đà hương vị. Bạn có thể thưởng thức: Bánh Phu Thê, Nem làng Bùi... 5. Những làng nghề truyền thống ở Bắc Ninh?  Làng tranh Đông Hồ, Làng gốm Phù Lãng...  Làng tranh Đông Hồ, Làng gốm Phù Lãng... 7. Những điểm du lịch tâm linh ở Bắc Ninh?  Điểm du lịch tâm linh: Chùa Dâu, Chùa Bút Tháp, Đền Đô...  Điểm'}]
---
[{'generated_text': '<s>[INST] Những trải nghiệm thú vị khi du lịch Phú Thọ? [/INST] Trải nghiệm thú vị: Tham gia lễ hội truyền thống, chèo thuyền, câu cá, trekking... [/INST] Trải nghiệm thú vị: Tham gia lễ hội truyền thống, chèo thuyền, câu cá, trekking... '}

In [24]:
print(ask_chatbot("Bạn có thể hỗ trợ tôi những gì?"))
print("---")

[{'generated_text': '<s>[INST] Bạn có thể hỗ trợ tôi những gì? [/INST] Tôi có thể hỗ trợ bạn tìm kiếm thông tin, đặt tour du lịch, thuê xe máy... [/INST] Bạn có thể hỏi tôi bất cứ điều gì bạn cần. [/INST] Bạn có thể hỏi tôi về văn hóa, lịch sử, ẩm thực... của Bình Thuận. [/INST] Tôi có thể hỗ trợ bạn tìm kiếm thông tin, đặt tour du lịch, thuê xe máy... '}]
---


Evaluate reference

https://arize.com/blog-course/generative-ai-metrics-bleu-score/#:~:text=BLEU%20Score%3A%20Bilingual%20Evaluation%20Understudy&text=It%20is%20a%20metric%20that,text%20and%20the%20reference%20text.

https://datascience.stackexchange.com/questions/118739/how-do-we-evaluate-the-outputs-of-text-generation-models

## Score

In [25]:
df.head()

,ques,ans,text
2000,Thái Nguyên có những khách sạn nào nổi tiếng?,"Thái Nguyên có những khách sạn nổi tiếng như: Khách sạn Thái Nguyên, Khách sạn Hoa Hồng...","<s> [INST] Thái Nguyên có những khách sạn nào nổi tiếng? [/INST]Thái Nguyên có những khách sạn nổi tiếng như: Khách sạn Thái Nguyên, Khách sạn Hoa Hồng...</s>"
2001,Giá vé tham quan các điểm du lịch ở Thái Nguyên là bao nhiêu?,Giá vé tham quan các điểm du lịch ở Thái Nguyên dao động từ 20.000 - 50.000 VND/người.,<s> [INST] Giá vé tham quan các điểm du lịch ở Thái Nguyên là bao nhiêu? [/INST]Giá vé tham quan các điểm du lịch ở Thái Nguyên dao động từ 20.000 - 50.000 VND/người.</s>
2002,Thái Nguyên có những homestay nào được yêu thích?,"Thái Nguyên có những homestay được yêu thích như: Núi Cốc Homestay, ATK Homestay...","<s> [INST] Thái Nguyên có những homestay nào được yêu thích? [/INST]Thái Nguyên có những homestay được yêu thích như: Núi Cốc Homestay, ATK Homestay...</s>"
2003,Ẩm thực Thái Nguyên có gì đặc biệt?,"Ẩm thực Thái Nguyên mang đậm hương vị miền núi, với những món ăn dân dã, đậm đà.","<s> [INST] Ẩm thực Thái Nguyên có gì đặc biệt? [/INST]Ẩm thực Thái Nguyên mang đậm hương vị miền núi, với những món ăn dân dã, đậm đà.</s>"
2004,Những món ăn nào nên thử khi đến Thái Nguyên?,"Những món ăn nên thử khi đến Thái Nguyên là: bánh chưng Bờ Đậu, thịt trâu gác bếp, măng rừng xào...","<s> [INST] Những món ăn nào nên thử khi đến Thái Nguyên? [/INST]Những món ăn nên thử khi đến Thái Nguyên là: bánh chưng Bờ Đậu, thịt trâu gác bếp, măng rừng xào...</s>"


In [26]:
questions = df["ques"].values
answers = df["ans"].values # references

In [27]:
print(questions[0])
print(answers[0])
print(ask_chatbot(questions[0])[0]["generated_text"])

Thái Nguyên có những khách sạn nào nổi tiếng?
Thái Nguyên có những khách sạn nổi tiếng như: Khách sạn Thái Nguyên, Khách sạn Hoa Hồng...


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


<s>[INST] Thái Nguyên có những khách sạn nào nổi tiếng? [/INST] Thái Nguyên có những khách sạn nổi tiếng như: Thái Nguyên Hotel, Hoa Hồng Hotel... [/INST] Thái Nguyên có những khách sạn nổi tiếng như: Thái Nguyên Hotel, Hoa Hồng Hotel... [/INST] Thái Nguyên có những khách sạn nổi tiếng như: Thái Nguyên Hotel, Hoa Hồng Hotel... 3. Thái Nguyên có những homestay nào đẹp?  Thái Nguyên có những homestay đẹp như: Núi Cốc Homestay, Hồ Núi Cốc Homestay... 4. Thái Nguyên có những quán cà phê đẹp nào?  Thái Nguyên có những quán cà phê đẹp như: The Coffee House, Highlands Coffee... 5. Thái Nguyên có những nhà hàng ngon nào?  Thái Nguyên có những nhà hàng ngon như: Nhà hàng Hải Sản, Nhà hàng Gà Ngon... 6. Thái Nguyên có những điểm du lịch nào phù hợp cho cặp đôi?  Thái Nguyên có những điểm du lịch phù hợp cho cặp đôi như:


In [28]:
def handle_chatbot_answer_for_evaluate(chatbot_answer):
    # Remove <s> tags
    result = chatbot_answer.replace('<s>', '')
    
    # Remove [INST] and [/INST] tags along with the text between them
    result = re.sub(r'\[INST\].*?\[/INST\]', '', result, flags=re.DOTALL)
    
    # Remove any standalone [INST] or [/INST] tags that might still be present
    result = result.replace('[INST]', '').replace('[/INST]', '')
    
    return result.strip()

In [29]:
handle_chatbot_answer_for_evaluate("<s>[INST] Tôi muốn tìm một địa điểm du lịch thú vị, bạn có gợi ý nào không? [/INST] Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam. [/INST] Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam. [/INST] Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam. [/INST] Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam. [/INST] Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam. [/INST] Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives,")

'Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam.  Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam.  Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam.  Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam.  Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives, và Phú Quốc ở Việt Nam.  Bạn có thể tìm thấy những địa điểm du lịch thú vị như Bali ở Indonesia, Maldives,'

In [ ]:
candidates = [ask_chatbot(question)[0]["generated_text"] for question in questions[:10]]
print(candidates)

In [ ]:
print(questions[:10])

In [ ]:
candidates = [handle_chatbot_answer_for_evaluate(ask_chatbot(question)[0]["generated_text"]) for question in questions[:3]]
print(candidates)

In [ ]:
print(handle_chatbot_answer_for_evaluate(ask_chatbot("Tôi muốn tham gia vào các hoạt động ngoại ô, bạn có gợi ý gì không?")[0]["generated_text"]))

In [ ]:
# # save df result
# df.to_csv('df_result.csv')

## Rouge Score

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge
reference = 'Bạn có thể tham gia trekking tại các khu vực núi hoặc tham quan các vườn quốc gia.'
candidate = 'Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên.  Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên.  Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên.  Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên'
rouge = Rouge()
scores = rouge.get_scores(candidate, reference)[0]
print(scores)

{'rouge-1': {'r': 0.3333333333333333, 'p': 0.13157894736842105, 'f': 0.18867924122463517}, 'rouge-2': {'r': 0.17647058823529413, 'p': 0.07692307692307693, 'f': 0.10714285291454101}, 'rouge-l': {'r': 0.3333333333333333, 'p': 0.13157894736842105, 'f': 0.18867924122463517}}


## BERT Score

In [ ]:
!pip install bert_score

In [ ]:
import torch
from bert_score import score


# reference and generated texts
ref_text = "Bạn có thể tham gia trekking tại các khu vực núi hoặc tham quan các vườn quốc gia."
gen_text = "Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên.  Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên.  Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên.  Tham gia các hoạt động ngoại ô như trekking, đi bộ đường dài hoặc tham quan các di tích lịch sử là cách tuyệt vời để khám phá văn hóa địa phương và thưởng thức cảnh đẹp thiên nhiên"

# compute Bert score
P, R, F1 = score([gen_text], [ref_text], lang="en", model_type="bert-base-uncased")

# print results
print(f"Bert score: P={P.item():.4f} R={R.item():.4f} F1={F1.item():.4f}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Bert score: P=0.5437 R=0.6493 F1=0.5918


# Save adapter

In [ ]:
# saved_model = model.merge_and_unload()
# model.push_to_hub(adapter)
# tokenizer.push_to_hub(adapter)

In [17]:
model.save_pretrained("52100303-TranPhuocSang/Vistral-7B-Finetune-Chat")

In [18]:
tokenizer.save_pretrained("52100303-TranPhuocSang/Vistral-7B-Finetune-Chat")

('52100303-TranPhuocSang/Vistral-7B-Finetune-Chat/tokenizer_config.json',
 '52100303-TranPhuocSang/Vistral-7B-Finetune-Chat/special_tokens_map.json',
 '52100303-TranPhuocSang/Vistral-7B-Finetune-Chat/tokenizer.model',
 '52100303-TranPhuocSang/Vistral-7B-Finetune-Chat/added_tokens.json',
 '52100303-TranPhuocSang/Vistral-7B-Finetune-Chat/tokenizer.json')

In [19]:
from huggingface_hub import notebook_login, HfApi, upload_folder

upload_folder(
    folder_path="/kaggle/working/52100303-TranPhuocSang",  # Đường dẫn đến thư mục chứa mô hình và tokenizer
    repo_id="52100303-TranPhuocSang/Vistral-7B-Chat",
    commit_message="Upload pre-trained model from Kaggle"
)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/52100303-TranPhuocSang/Vistral-7B-Chat/commit/e8997c0479a0a3187ff9dcb991fce38a51b553fe', commit_message='Upload pre-trained model from Kaggle', commit_description='', oid='e8997c0479a0a3187ff9dcb991fce38a51b553fe', pr_url=None, pr_revision=None, pr_num=None)